In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO # Or A2C, DQN for discrete actions
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv
import time
from stable_baselines3.common.callbacks import ProgressBarCallback
from stable_baselines3.common.vec_env import SubprocVecEnv


pybullet build time: Jan 29 2025 23:16:28


In [2]:
class MazeCarEnv(gym.Env):
    metadata = {'render_modes': ['human'], "render_fps": 30}

    def __init__(self, render_mode=None):
        super().__init__()

        # --- Define Action Space ---
        # Hier exemplarisch: 2 diskrete Aktionen (links/rechts drehen + vorrücken)
        self.action_space = spaces.Discrete(2)  # 0: links, 1: rechts
        
        # --- Define Observation Space ---
        # Beobachtung: [Auto_x, Auto_y, Auto_yaw, Ziel_x, Ziel_y]
        low = np.array([-6, -6, -np.pi, -6, -6], dtype=np.float32)
        high = np.array([6, 6, np.pi, 6, 6], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # --- PyBullet Setup ---
        self.render_mode = render_mode
        self.client = p.connect(p.DIRECT if render_mode is None else p.GUI)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81, physicsClientId=self.client)

        # Lade eine Plane als Boden
        self.planeId = p.loadURDF("plane.urdf", physicsClientId=self.client)

        # Maze als URDF laden (dieses sollte z. B. als "maze.urdf" im Arbeitsverzeichnis liegen)
        # Die Option useFixedBase=True sorgt dafür, dass das Maze statisch bleibt.
        self.mazeId = p.loadURDF("urdf/maze.urdf",
                                 basePosition=[0, 0, 0],
                                 useFixedBase=True,
                                 physicsClientId=self.client)

        # Ziele definieren (2 verschiedene, beispielhaft)
        # Passe sie an die tatsächlichen Gänge/Ausgänge in deinem Maze an.
        self.goal_area_1 = np.array([4.5, 4.5])   # oberer Ausgang
        self.goal_area_2 = np.array([4.5, -4.5])  # unterer Ausgang
        self.goal_radius = 0.5
        self.target_goal_pos = None  # Wird in reset() gesetzt
        self.correct_goal_index = -1

        # Falls im GUI-Modus: Visualisiere die Ziele als farbige Kugeln
        if p.getConnectionInfo()['connectionMethod'] == p.GUI:
            goal_visual_shape_1 = p.createVisualShape(
                p.GEOM_SPHERE,
                radius=self.goal_radius,
                rgbaColor=[0, 1, 0, 0.5]  # halbdurchsichtig grün
            )
            goal_visual_shape_2 = p.createVisualShape(
                p.GEOM_SPHERE,
                radius=self.goal_radius,
                rgbaColor=[1, 0, 0, 0.5]  # halbdurchsichtig rot
            )
            p.createMultiBody(baseVisualShapeIndex=goal_visual_shape_1,
                              basePosition=[self.goal_area_1[0], self.goal_area_1[1], 0.1])
            p.createMultiBody(baseVisualShapeIndex=goal_visual_shape_2,
                              basePosition=[self.goal_area_2[0], self.goal_area_2[1], 0.1])

        # Roboter (Auto) laden
        # Passe ggf. den Pfad an, wenn das URDF woanders liegt
        self.start_pos = [-4.5, 0.0, 0.1]  # Start nahe der linken Seite des Maze
        self.start_orn = p.getQuaternionFromEuler([0, 0, 0])
        self.carId = p.loadURDF("urdf/simple_two_wheel_car.urdf",
                                self.start_pos, self.start_orn,
                                physicsClientId=self.client)
        
        # IDs der Räder (müssen zu deinem Roboter-URDF passen)
        self.left_wheel_joint_index = 1
        self.right_wheel_joint_index = 0

        self.step_counter = 0
        self.max_steps_per_episode = 500  # z. B. mehr Schritte erlauben

        self.action_repeat = 25

    def _get_obs(self):
        pos, orn_quat = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        euler = p.getEulerFromQuaternion(orn_quat)
        yaw = euler[2]
        return np.array([pos[0], pos[1], yaw,
                         self.target_goal_pos[0],
                         self.target_goal_pos[1]], dtype=np.float32)

    def _get_info(self):
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        dist_goal1 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_1)
        dist_goal2 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_2)
        return {
            "distance_goal1": dist_goal1,
            "distance_goal2": dist_goal2,
            "target_goal_index": self.correct_goal_index
        }

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.step_counter = 0

        # Auto zurücksetzen
        p.resetBasePositionAndOrientation(self.carId, self.start_pos, self.start_orn, physicsClientId=self.client)
        p.resetBaseVelocity(self.carId,
                            linearVelocity=[0, 0, 0],
                            angularVelocity=[0, 0, 0],
                            physicsClientId=self.client)

        # Zufällige Wahl des "richtigen" Ziels (oder fest vorgegeben)
        # self.correct_goal_index = self.np_random.integers(0, 2)
        self.correct_goal_index = 0  # zum Testen immer Goal 1
        if self.correct_goal_index == 0:
            self.target_goal_pos = self.goal_area_1
        else:
            self.target_goal_pos = self.goal_area_2

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()
            # time.sleep(0.02)

        return observation, info

    def step(self, action):

        if action == 0:
            left_vel, right_vel = 10.0, 10.0
        elif action == 1:
            left_vel, right_vel = 2.0, 10.0
        else:  # action == 2
            left_vel, right_vel = 10.0, 2.0

        # Setze die Motoren der beiden Räder
        p.setJointMotorControl2(
            bodyUniqueId=self.carId,
            jointIndex=self.left_wheel_joint_index,
            controlMode=p.VELOCITY_CONTROL,
            targetVelocity=left_vel,
            force=20.0
        )
        p.setJointMotorControl2(
            bodyUniqueId=self.carId,
            jointIndex=self.right_wheel_joint_index,
            controlMode=p.VELOCITY_CONTROL,
            targetVelocity=right_vel,
            force=20.0
        )


        # print(f"Action: {action}, Velocities: L={left_vel}, R={right_vel}, Step={self.step_counter}")
        # car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        # print(f"Position: {car_pos}")


        # Jetzt die Simulation mehrmals updaten,
        # damit der Roboter tatsächlich fährt und gegen Wände kollidieren kann
        for _ in range(self.action_repeat):
            p.stepSimulation()
            # if self.render_mode == "human":
            #     time.sleep(0.001) 
            #     print("Step Simulation")
        
        self.step_counter += 1

        # Beobachtung + Reward + Done bestimmen
        observation = self._get_obs()
        info = self._get_info()

        # Check goal conditions
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        in_goal_1 = np.linalg.norm(car_pos[:2] - self.goal_area_1) < self.goal_radius
        in_goal_2 = np.linalg.norm(car_pos[:2] - self.goal_area_2) < self.goal_radius

        terminated = False
        reward = -0.01  # kleiner Schritt-Penalty

        if in_goal_1:
            if self.correct_goal_index == 0:
                reward = 10.0
                terminated = True
            else:
                reward = -5.0
                terminated = True
        elif in_goal_2:
            if self.correct_goal_index == 1:
                reward = 10.0
                terminated = True
            else:
                reward = -5.0
                terminated = True

        truncated = (self.step_counter >= self.max_steps_per_episode)

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, truncated, info

    def render(self):
        # Bei PyBullet im GUI-Modus passiert das Rendering automatisch
        pass

    def _render_frame(self):
        # Debug-Anzeigen, falls erwünscht
        pass

    def close(self):
        p.disconnect(physicsClientId=self.client)


In [3]:
def make_env():
    def _init():
        return MazeCarEnv(render_mode=None)  # or "human" for visualization
    return _init


In [4]:
num_envs = 4
env = SubprocVecEnv([make_env() for _ in range(num_envs)])
# env = MazeCarEnv(render_mode=None) 
# env = MazeCarEnv(render_mode="human")

# try:
#     check_env(env)
#     print("Environment check passed!")
# except Exception as e:
#     print(f"Environment check failed: {e}")
#     env.close()
#     exit()

# model = SAC(
#     "MlpPolicy", 
#     env, 
#     verbose=1, 
#     tensorboard_log="./sac_mazecar_tensorboard/",
#     batch_size=64,
#     learning_rate=0.0003,
#     train_freq=1,
#     gradient_steps=1,
#     buffer_size=1000000,
#     tau=0.005,
#     gamma=0.99
# )

n_steps_calc = num_envs * 512

model = PPO("MlpPolicy", 
            env, 
            verbose=1, 
            tensorboard_log="./ppo_mazecar_tensorboard/",
            batch_size=256, # after n_steps the data is split into batches
            learning_rate=0.0003,
            n_steps=512,   # number of steps collected before a training
            device="cuda"
)


pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28


Using cuda device


/home/ferdinand/masterthesis/ve_pybullet/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [ ]:
# # Train the model
# check_env = DummyVecEnv([lambda: MazeCarEnv(render_mode="human")])
# check_callback = EvalCallback(
#     check_env, 
#     best_model_save_path='./logs/',
#     log_path='./logs/', 
#     eval_freq=100,
#     deterministic=True, 
#     render=True
# )


model.learn(total_timesteps=200000, progress_bar=True)
# model.learn(total_timesteps=5000)

model.save("ppo_mazecar_model_2_multi")
env.close()
# check_env.close()

Logging to ./ppo_mazecar_tensorboard/PPO_55


Output()

-----------------------------
| time/              |      |
|    fps             | 962  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 1021         |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0014756119 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.692       |
|    explained_variance   | -36.3        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0145      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00103     |
|    value_loss           | 0.00341      |
------------------------------------------


----------------------------------------
| time/                   |            |
|    fps                  | 1052       |
|    iterations           | 3          |
|    time_elapsed         | 5          |
|    total_timesteps      | 6144       |
| train/                  |            |
|    approx_kl            | 0.01787328 |
|    clip_fraction        | 0.0927     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.684     |
|    explained_variance   | -0.311     |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0131    |
|    n_updates            | 20         |
|    policy_gradient_loss | -0.0052    |
|    value_loss           | 0.000293   |
----------------------------------------


...

------------------------------------------
| time/                   |              |
|    fps                  | 1101         |
|    iterations           | 94           |
|    time_elapsed         | 174          |
|    total_timesteps      | 192512       |
| train/                  |              |
|    approx_kl            | 0.0033323949 |
|    clip_fraction        | 0.0168       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.237       |
|    explained_variance   | 0.953        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00442     |
|    n_updates            | 930          |
|    policy_gradient_loss | -0.00291     |
|    value_loss           | 9.78e-05     |
------------------------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 1098         |
|    iterations           | 95           |
|    time_elapsed         | 177          |
|    total_timesteps      | 194560       |
| train/                  |              |
|    approx_kl            | 0.0027433499 |
|    clip_fraction        | 0.0111       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.206       |
|    explained_variance   | 0.967        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00678     |
|    n_updates            | 940          |
|    policy_gradient_loss | -0.00188     |
|    value_loss           | 3.58e-05     |
------------------------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 1097         |
|    iterations           | 96           |
|    time_elapsed         | 179          |
|    total_timesteps      | 196608       |
| train/                  |              |
|    approx_kl            | 0.0015068908 |
|    clip_fraction        | 0.00933      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.226       |
|    explained_variance   | 0.914        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.00021      |
|    n_updates            | 950          |
|    policy_gradient_loss | -0.00145     |
|    value_loss           | 3.54e-05     |
------------------------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 1096         |
|    iterations           | 97           |
|    time_elapsed         | 181          |
|    total_timesteps      | 198656       |
| train/                  |              |
|    approx_kl            | 0.0032570502 |
|    clip_fraction        | 0.0525       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.28        |
|    explained_variance   | 0.934        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00746     |
|    n_updates            | 960          |
|    policy_gradient_loss | -0.0024      |
|    value_loss           | 5.31e-05     |
------------------------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 1094         |
|    iterations           | 98           |
|    time_elapsed         | 183          |
|    total_timesteps      | 200704       |
| train/                  |              |
|    approx_kl            | 0.0013758931 |
|    clip_fraction        | 0.00933      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.308       |
|    explained_variance   | 0.953        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00378     |
|    n_updates            | 970          |
|    policy_gradient_loss | -0.0008      |
|    value_loss           | 3.39e-05     |
------------------------------------------


In [ ]:
# --- To evaluate ---
eval_env = MazeCarEnv(render_mode="human") # Render during evaluation
model = PPO.load("ppo_mazecar_model_2_multi", env=eval_env)

obs, info = eval_env.reset()
for _ in range(200): # Max steps for evaluation
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)
    time.sleep(1./120.0)  # Adjust sleep time for rendering speed
    if terminated or truncated:
        print(f"Evaluation episode finished. Reached target goal: {info.get('target_goal_index')}, Reward: {reward}")
        obs, info = eval_env.reset() # Reset for next evaluation episode

eval_env.close()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RPL-P)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
Wrapping the env

In [7]:
env.close()

check_env.close()

eval_env.close()


AttributeError: 'function' object has no attribute 'close'

In [ ]:
# ~/masterthesis/mt_start$   tensorboard --logdir=./ppo_mazecar_tensorboard/

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.get_device_name(0))  # Prints the name of the GPU

True
NVIDIA GeForce RTX 4060 Laptop GPU
